In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/English_OCR_dataset/image_test.zip

Streaming output truncated to the last 5000 lines.
  inflating: image_test/k/img021-035.png  
  inflating: image_test/k/img021-042.png  
  inflating: image_test/k/img021-047.png  
  inflating: image_test/k/img021-052.png  
  inflating: image_test/k/img047-00001.png  
  inflating: image_test/k/img047-00007.png  
  inflating: image_test/k/img047-00011.png  
  inflating: image_test/k/img047-00021.png  
  inflating: image_test/k/img047-00025.png  
  inflating: image_test/k/img047-00032.png  
  inflating: image_test/k/img047-00035.png  
  inflating: image_test/k/img047-00036.png  
  inflating: image_test/k/img047-00045.png  
  inflating: image_test/k/img047-00049.png  
  inflating: image_test/k/img047-00057.png  
  inflating: image_test/k/img047-00059.png  
  inflating: image_test/k/img047-00061.png  
  inflating: image_test/k/img047-00071.png  
  inflating: image_test/k/img047-00073.png  
  inflating: image_test/k/img047-00076.png  
  inflating: image_test/k/img047-00081.png  
  inflating:

In [3]:
!unzip /content/drive/MyDrive/English_OCR_dataset/image_train.zip

Streaming output truncated to the last 5000 lines.
  inflating: image_train/x/img034-00547.png  
  inflating: image_train/x/img034-00548.png  
  inflating: image_train/x/img034-00551.png  
  inflating: image_train/x/img034-00552.png  
  inflating: image_train/x/img034-00553.png  
  inflating: image_train/x/img034-00554.png  
  inflating: image_train/x/img034-00555.png  
  inflating: image_train/x/img034-00556.png  
  inflating: image_train/x/img034-00557.png  
  inflating: image_train/x/img034-00558.png  
  inflating: image_train/x/img034-00559.png  
  inflating: image_train/x/img034-00560.png  
  inflating: image_train/x/img034-00561.png  
  inflating: image_train/x/img034-00562.png  
  inflating: image_train/x/img034-00563.png  
  inflating: image_train/x/img034-00565.png  
  inflating: image_train/x/img034-00566.png  
  inflating: image_train/x/img034-00567.png  
  inflating: image_train/x/img034-00569.png  
  inflating: image_train/x/img034-00570.png  
  inflating: image_train/x/im

In [4]:
ls

drive/  image_test/  image_train/  sample_data/


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

train_dir = './image_train'
test_dir = './image_test'

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
vgg.trainable = False 

model = Sequential()
model.add(vgg)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(len(train_generator.class_indices), activation='softmax'))  # تعداد خروجی برابر با تعداد کلاس‌ها


model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


checkpoint_callback = ModelCheckpoint(
    filepath='/content/drive/MyDrive/English_OCR_dataset/model_checkpoint.weights.h5',
    save_weights_only=True,
    save_best_only=True,
    verbose=1
)


try:
    model.load_weights('/content/drive/MyDrive/English_OCR_dataset/model_checkpoint.weights.h5')
    print("Weights loaded successfully.")
except:
    print("No saved weights found. Starting training from scratch.")


model.fit(
    train_generator,
    epochs=10,              
    initial_epoch=5,        
    validation_data=test_generator,
    callbacks=[checkpoint_callback]
)


Found 54846 images belonging to 36 classes.
Found 11556 images belonging to 36 classes.
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
No saved weights found. Starting training from scratch.
Epoch 6/10


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 62 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1714/1714 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - accuracy: 0.9252 - loss: 0.3116
Epoch 6: val_loss improved from inf to 0.07657, saving model to /content/drive/MyDrive/English_OCR_dataset/model_checkpoint.weights.h5
1714/1714 ━━━━━━━━━━━━━━━━━━━━ 283s 156ms/step - accuracy: 0.9253 - loss: 0.3115 - val_accuracy: 0.9728 - val_loss: 0.0766
Epoch 7/10
1714/1714 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 0.9875 - loss: 0.0329
Epoch 7: val_loss improved from 0.07657 to 0.06243, saving model to /content/drive/MyDrive/English_OCR_dataset/model_checkpoint.weights.h5
1714/1714 ━━━━━━━━━━━━━━━━━━━━ 298s 150ms/step - accuracy: 0.9875 - loss: 0.0329 - val_accuracy: 0.9790 - val_loss: 0.0624
Epoch 8/10
1714/1714 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 0.9899 - loss: 0.0268
Epoch 8: val_loss improved from 0.06243 to 0.05457, saving model to /content/drive/MyDrive/English_OCR_dataset/model_checkpoint.weights.h5
1714/1714 ━━━━━━━━━━━━━━━━━━━━ 267s 155ms/step - accuracy: 0.9899 - loss: 0.0269 -

In [20]:
model.save('/content/drive/MyDrive/English_OCR_dataset/model2.h5')

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np


def predict_image(img_path, model):
 
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  
    img_array /= 255.0  


    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=1)[0]  

    
    class_labels = {v: k for k, v in train_generator.class_indices.items()}  
    print(f"Predicted class: {class_labels[predicted_class]}")


img_path = '/content/image_test/t/img030-00030.png' 
predict_image(img_path, model)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Predicted class: t
